In [1]:
import pickle
import datetime
import numpy as np
import Dataset
import CalZeu

In [2]:
with open("../0 save inner data/2 sorted_pigments_all.pkl", 'rb') as f:
    SOURCES, DATES, LONS, LATS, DEPTHS, PIGMENTS, USE_PIGMENTS = pickle.load(f)

In [3]:
SOURCES, DATES, LONS, LATS, DEPTHS = np.array(SOURCES), np.array(DATES), np.array(LONS), np.array(LATS), np.array(DEPTHS)
for var in PIGMENTS: PIGMENTS[var] = np.array(PIGMENTS[var])
for var in USE_PIGMENTS: USE_PIGMENTS[var] = np.array(USE_PIGMENTS[var])

In [4]:
dataset = {}

for i in range(len(SOURCES)):
    mk = "{:0>4d}\t{:0>2d}\t{:0>2d}\t{:0>4d}\t{:0>4d}\t{:0>3d}".format(DATES[i].year,
                                                                       DATES[i].month,
                                                                       DATES[i].day,
                                                                       int(np.round(LATS[i]*100)),
                                                                       int(np.round(LONS[i]*100)),
                                                                       int(np.round(DEPTHS[i])))
    if mk not in dataset:
        dataset[mk] = Dataset.Dataset(date=datetime.datetime(year=DATES[i].year,
                                                             month=DATES[i].month,
                                                             day=DATES[i].day),
                                      lon=np.round(LONS[i],2),
                                      lat=np.round(LATS[i],2),
                                      depth=np.round(DEPTHS[i]))

    dataset[mk].add_index(i)

In [5]:
for mk in dataset:
    index = np.array(dataset[mk].index)
    # source
    dataset[mk].add_source(list(  np.sort(np.unique(SOURCES[index]))  ))
    # pigments&pigments_use
    dataset[mk].add_pigments(  {p:np.nanmedian(PIGMENTS[p][index][USE_PIGMENTS[p][index]]) for p in PIGMENTS}  )
    dataset[mk].add_pigments_use(  {p:~np.isnan(dataset[mk].pigments[p]) for p in dataset[mk].pigments}  )

C:\Users\yy\AppData\Roaming\Python\Python39\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [6]:
# with open("I:/satellite match data/SSTday.pkl",'rb') as f:
#     SSTday = pickle.load(f)   
# with open("I:/satellite match data/SSTnight.pkl",'rb') as f:
#     SSTnight = pickle.load(f)
    
with open("I:/satellite match data/sal.pkl",'rb') as f:
    sal = pickle.load(f) 
with open("I:/satellite match data/tem.pkl",'rb') as f:
    tem = pickle.load(f) 
with open("I:/satellite match data/uo.pkl",'rb') as f:
    uo = pickle.load(f) 
with open("I:/satellite match data/vo.pkl",'rb') as f:
    vo = pickle.load(f)
with open("I:/satellite match data/ssh.pkl",'rb') as f:
    ssh = pickle.load(f)
with open("I:/satellite match data/mld.pkl",'rb') as f:
    mld = pickle.load(f)
    
with open("I:/satellite match data/Rrs412.pkl",'rb') as f:
    Rrs412 = pickle.load(f)  
with open("I:/satellite match data/Rrs443.pkl",'rb') as f:
    Rrs443 = pickle.load(f)  
with open("I:/satellite match data/Rrs490.pkl",'rb') as f:
    Rrs490 = pickle.load(f)  
with open("I:/satellite match data/Rrs510.pkl",'rb') as f:
    Rrs510 = pickle.load(f)  
with open("I:/satellite match data/Rrs560.pkl",'rb') as f:
    Rrs560 = pickle.load(f)  
with open("I:/satellite match data/Rrs665.pkl",'rb') as f:
    Rrs665 = pickle.load(f) 
with open("I:/satellite match data/chla.pkl",'rb') as f:
    chla = pickle.load(f)  

    
# with open("I:/0 save inner data/satellite match data/sla.pkl",'rb') as f:
#     sla = pickle.load(f)
# with open("I:/0 save inner data/satellite match data/ugos.pkl",'rb') as f:
#     ugos = pickle.load(f)
# with open("I:/0 save inner data/satellite match data/vgos.pkl",'rb') as f:
#     vgos = pickle.load(f)   
    
print(sal.shape)

(86327, 15, 3, 3)


In [7]:
z1 = []
z05 = []

# satellite_match&satellite_match_use
for mk in dataset:
    index = dataset[mk].index[0]
    sate_match = {
                  "tem":tem[index],
                  "sal":sal[index],
                  "uo":uo[index],
                  "vo":vo[index],
                  "mld":mld[index],
                  "ssh":ssh[index],
#                   "SSTday":SSTday[index]-273.15,
#                   "SSTnight":SSTnight[index]-273.15,
                  "Rrs412":Rrs412[index],
                  "Rrs443":Rrs443[index],
                  "Rrs490":Rrs490[index],
                  "Rrs510":Rrs510[index],
                  "Rrs560":Rrs560[index],
                  "Rrs665":Rrs665[index],
        
                  "chla":chla[index],          
                  "Zeu":CalZeu.Zeu(Rrs440=Rrs443[index],Rrs490=Rrs490[index],Rrs555=Rrs560[index],option="IOP",tE='1%'),
#                   "Zeu":CalZeu.Zeu(chla=chla[index],option="Chl")
#                   "sla":sla[index],
#                   "ugos":ugos[index],
#                   "vgos":vgos[index],
                  }
    z1.append(CalZeu.Zeu(Rrs440=Rrs443[index],Rrs490=Rrs490[index],Rrs555=Rrs560[index],option="IOP",tE='1%'))
    z05.append(CalZeu.Zeu(Rrs440=Rrs443[index],Rrs490=Rrs490[index],Rrs555=Rrs560[index],option="IOP",tE='0.5%'))
    sate_match_use = {}
    for item in sate_match:
        sate_match_item = sate_match[item].reshape(sate_match[item].shape[0],-1)
        sate_match[item] = np.nanmean(sate_match_item,axis=1)
        use = (~np.isnan(sate_match[item])).sum()>=1 # len(sate_match[item])*0.3
        sate_match_use[item] = use
        
#     case1 = (np.nanmean(sate_match["Rrs412"])/np.nanmean(sate_match["Rrs443"]))>=1.0
#     for item in sate_match_use:
#         sate_match_use[item] = sate_match_use[item]&case1
    
    # strict case1
    RR12 = np.nanmean(sate_match["Rrs412"])/np.nanmean(sate_match["Rrs443"])
    RR53 = np.nanmean(sate_match["Rrs560"])/np.nanmean(sate_match["Rrs490"])
    RR12_CS1 = 0.9351+0.113/RR53-0.0217/(RR53**2)+0.003/(RR53**3)
    Rrs555_CS1 = 0.0006+0.0027*RR53-0.0004*RR53**2-0.0002*RR53**3
    gamma,nu = 0.1,0.5
    strict_case1 = ((1-gamma)*RR12_CS1<=RR12)&(RR12<=(1+gamma)*RR12_CS1)
    strict_case1 = ((1-nu)*Rrs555_CS1<=np.nanmean(sate_match["Rrs560"]))&(np.nanmean(sate_match["Rrs560"])<=(1+nu)*Rrs555_CS1)
    for item in sate_match_use:
        sate_match_use[item] = sate_match_use[item]&strict_case1
     
    dataset[mk].add_satellite_match(sate_match)
    dataset[mk].add_satellite_match_use(all([sate_match_use[item] for item in sate_match_use]))
    

C:\Users\yy\AppData\Local\Temp\ipykernel_24564\3844542987.py:35: RuntimeWarning: Mean of empty slice
  sate_match[item] = np.nanmean(sate_match_item,axis=1)
C:\Users\yy\AppData\Local\Temp\ipykernel_24564\3844542987.py:44: RuntimeWarning: Mean of empty slice
  RR12 = np.nanmean(sate_match["Rrs412"])/np.nanmean(sate_match["Rrs443"])
C:\Users\yy\AppData\Local\Temp\ipykernel_24564\3844542987.py:45: RuntimeWarning: Mean of empty slice
  RR53 = np.nanmean(sate_match["Rrs560"])/np.nanmean(sate_match["Rrs490"])
C:\Users\yy\AppData\Local\Temp\ipykernel_24564\3844542987.py:50: RuntimeWarning: Mean of empty slice
  strict_case1 = ((1-nu)*Rrs555_CS1<=np.nanmean(sate_match["Rrs560"]))&(np.nanmean(sate_match["Rrs560"])<=(1+nu)*Rrs555_CS1)


In [8]:
for mk in dataset:
    dataset[mk].revise_use()

In [9]:
with open("../0 save inner data/dataset_all.pkl",'wb') as f:
    pickle.dump(dataset, f)

In [10]:
z001 = np.stack(z1,axis=0).flatten()
z0005 = np.stack(z05,axis=0).flatten()

nan = np.isnan(z001)|np.isnan(z0005)|(z001>360)
z001 = z001[~nan]
z0005 = z0005[~nan]


In [11]:
import scipy.io as scio
scio.savemat('z001_z0005.mat',{'z1':z001, 'z05':z0005})